In [1]:
import tiktoken
import pandas as pd
from datetime import datetime

df = pd.read_csv('airline_review.csv',index_col=0)
df.fillna("Unknown",inplace=True)
print(df.shape)
df.head()

(3616, 19)


,rating,header,author,date,place,content,aircraft,traveller_type,seat_type,route,date_flown,seat_comfort,cabin_staff_service,food_beverages,ground_service,value_for_money,recommended,entertainment,trip_verified
0,2,service was mediocre at best,Gary Storer,2023-10-03,United Kingdom,"Just returned from Chicago, flew out 10 days ...",A380,Couple Leisure,Economy Class,Chicago to Manchester via Heathrow,2023-10-01,2,3,1,2,2,no,-1,Not Verified
1,2,BA standards continue to decline,A Jensen,2023-10-02,United Kingdom,BA standards continue to decline every time ...,A320,Business,Business Class,London Heathrow to Munich,2023-09-01,2,1,2,1,1,no,-1,Verified
2,2,"won the race to the bottom""",John Rockett,2023-10-02,United Kingdom,Awful. Business class check in queue just as...,A320,Couple Leisure,Business Class,Heathrow to Istanbul,2023-09-01,2,3,2,1,1,no,-1,Not Verified
3,3,Not a reliable airline,Tatiana Bobrovskaya,2023-10-02,United Kingdom,Not a reliable airline. You cannot trust the...,A320,Business,Economy Class,London to Geneva,2023-10-01,4,4,2,1,1,no,-1,Verified
4,1,It is a national disgrace,A Dawson,2023-09-30,United Kingdom,I take comfort in reading the last ten or so...,Unknown,Couple Leisure,Business Class,Athens to London,2023-09-01,1,1,1,1,1,no,1,Verified


In [2]:
def encode_column(column, encoding):
    return column.apply(lambda x: encoding[x] if x in encoding and x != "Unknown" else x)

"""
GPT-4 was used to create these labels to be used in the embeddings via this prompt: 
   - For each category in airline review data with ratings (-1, 1, 2, 3, 4, 5) 
     provide one to two word labels that are infmorative to the customers expirience 
"""

seat_comfort_encoding = {
    -1: "Unbearable",
    1: "Uncomfortable",
    2: "Tolerable",
    3: "Adequate",
    4: "Comfortable",
    5: "Luxurious"
}

cabin_service_encoding = {
    -1: "Negligent",
    1: "Inadequate",
    2: "Basic",
    3: "Attentive",
    4: "Exceptional",
    5: "Outstanding"
}

entertainment_encoding = {
    -1: "Non-Existent",
    1: "Limitted",
    2: "Adequate",
    3: "Good",
    4: "Excellent",
    5: "Supberb"
}

ground_service_encoding ={
    -1: "Dismissive",
    1: "Unhelpful",
    2: "Sufficient",
    3: "Helpful",
    4: "Effecient",
    5: "Exceptional"
}

value_for_money_encoding = {
    -1: "Overpriced",
    1: "Expensive",
    2: "Fair",
    3: "Reasonable",
    4: "Good Value",
    5: "Excellent Value"
}

df['seat_comfort'] = encode_column(df['seat_comfort'],seat_comfort_encoding)
df['cabin_staff_service'] = encode_column(df['cabin_staff_service'],cabin_service_encoding)
df['entertainment'] = encode_column(df['entertainment'],entertainment_encoding)
df['ground_service'] = encode_column(df['ground_service'],ground_service_encoding)
df['value_for_money'] = encode_column(df['value_for_money'],value_for_money_encoding)

In [3]:
def convert_date_format(date_str):
    """
    Converted date from 'YYYY-MM-DD' format to 'DayOfWeek, MonthName Day(th/st/nd/rd), Year' format.
    Skips conversion for rows where the value is "Unknown". This format is better suited for embeddings.
    """
    if date_str == "Unknown":
        return "Unknown"

    try:
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        day_suffix = lambda n: 'th' if 11 <= n <= 13 else {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
        return date_obj.strftime(f"%A, %B {date_obj.day}{day_suffix(date_obj.day)}, %Y")
    except ValueError:
        # Return original string if not a valid date
        return date_str

df['date_flown'] = df['date_flown'].astype(str)

df['date_flown_formatted'] = df['date_flown'].apply(convert_date_format)

In [4]:
df['embedding_input'] = (
    "CUSTOMER INFORMATION - " + 
    "AIRCRAFT: " + df.aircraft.str.strip() +
    "; TRAVELER TYPE: " + df.traveller_type.str.strip() +
    "; SEAT TYPE: " + df.seat_type.str.strip() +
    "; ROUTE: " + df.route.str.strip() +
    "; DATE FLOWN: " + df.date_flown_formatted.str.strip() +
    ". REVIEW INFORMATION - " +
    "REVIEW TITLE: " + df.header.str.strip() +
    "; REVIEW CONTENT: " + df.content.str.strip() +
    "; SEAT COMFORT: " + df.seat_comfort.str.strip() +
    "; CABIN STAFF SERVICE: " + df.cabin_staff_service.str.strip() +
    "; ENTERTAINMENT: " + df.entertainment.str.strip() +
    "; GROUND SERVICE: " + df.ground_service.str.strip() +
    "; VALUE FOR MONEY: " + df.value_for_money.str.strip() +
    "; RECOMMENDED: " + df.recommended.str.strip()
)

In [5]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


# Apply the function to each row in the 'content' column
def count_tokens(row):
    return num_tokens_from_string(row['embedding_input'], 'cl100k_base')

# Add a new column 'num_tokens' to store the token count for each row
df['num_tokens'] = df.apply(count_tokens, axis=1)

# Get the total token count
total_tokens = df['num_tokens'].sum()

print(total_tokens)

1081077


In [6]:
embedding_cost = (total_tokens / 1000) * .00013

print(embedding_cost) 

# The cost of embedding all of the rows with the text-embedding-3-large model

0.14054001


In [7]:
df.to_csv('processed_airline_reviews.csv', index=False)